<a href="https://colab.research.google.com/github/OleksandrZadvornyi/weather-forecasting/blob/main/prepare_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q dask
!pip install -q gluonts ujson
!pip install -q datasets

In [ ]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import shutil
from datasets import Dataset, Features, Value, Sequence, DatasetDict
from gluonts.itertools import Map

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# When reading the CSV files, specify dtypes
dtypes = {
    'STATION': 'object',
    'DATE': 'object',  # Start as string, convert to datetime later
    'LATITUDE': 'float64',
    'LONGITUDE': 'float64',
    'ELEVATION': 'float64',
    'NAME': 'object',
    'PRCP': 'object',  # Keep as object initially to handle missing values
    'PRCP_ATTRIBUTES': 'object',
    'TMAX': 'object',
    'TMAX_ATTRIBUTES': 'object',
    'TMIN': 'object',
    'TMIN_ATTRIBUTES': 'object',
    'TAVG': 'object',
    'TAVG_ATTRIBUTES': 'object',
    'SNWD': 'object',
    'SNWD_ATTRIBUTES': 'object',
}

# 1. Load Ukrainian station data
ukrainian_files = glob.glob('/content/drive/MyDrive/weather_forecasting_project/daily-summaries-latest-upm/*.csv')
print(f"Found {len(ukrainian_files)} data files")

# Set files count limit
max_files = 20
ukrainian_files = ukrainian_files[:max_files]
print(f"Reading {len(ukrainian_files)} files")

In [ ]:
# 2. Read and combine files
ddf = dd.read_csv(ukrainian_files, dtype=dtypes, assume_missing=True)
print(f"Data loaded with {len(ddf.columns)} columns")

In [ ]:
# 3. Apply preprocessing to sample for metadata
def preprocess(df):
    df = df.copy()

    df['DATE'] = pd.to_datetime(df['DATE'])

    # Ensure all expected columns exist, filling with NaN if necessary
    expected_columns = ['PRCP', 'TMAX', 'TMIN', 'TAVG']
    for col in expected_columns:
        if col not in df.columns:
            df[col] = np.nan  # Ensure column exists
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Convert temperature from tenths of degrees Celsius to Celsius
    temperature_columns = ['TMAX', 'TMIN', 'TAVG']
    for col in temperature_columns:
        if col in df.columns:
            df[col] = df[col] / 10.0

    if 'TMAX' in df.columns and 'TMIN' in df.columns:
        df['TEMP_RANGE'] = df['TMAX'] - df['TMIN']

    df['MONTH'] = df['DATE'].dt.month
    df['DAY'] = df['DATE'].dt.day
    df['YEAR'] = df['DATE'].dt.year
    df['DAY_OF_YEAR'] = df['DATE'].dt.dayofyear
    df['SEASON'] = ((df['MONTH'] % 12) // 3 + 1).astype(int)

    cols_to_drop = [col for col in df.columns if '_ATTRIBUTES' in col]
    df = df.drop(columns=cols_to_drop, errors='ignore')

    # Ensure column order matches the metadata
    expected_order = ['STATION', 'DATE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME',
                      'PRCP', 'TMAX', 'TMIN', 'TAVG', 'TEMP_RANGE',
                      'MONTH', 'DAY', 'YEAR', 'DAY_OF_YEAR', 'SEASON']

    df = df.reindex(columns=expected_order)

    return df

def process_station_data(station_df):
    station_df = station_df.sort_values('DATE')

    if len(station_df) < 365:
        return None

    station_meta = station_df.iloc[0][['STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION']].to_dict()

    station_df = station_df.set_index('DATE').loc[~station_df.index.duplicated(keep='first')]

    date_range = pd.date_range(start=station_df.index.min(), end=station_df.index.max(), freq='D')
    resampled_df = station_df.reindex(date_range)

    numeric_cols = ['PRCP', 'TMAX', 'TMIN', 'TAVG', 'TEMP_RANGE']
    numeric_cols = [col for col in numeric_cols if col in resampled_df.columns]

    def seasonal_interpolation(series):
        interpolated = series.interpolate(method='linear', limit=7)
        return interpolated.fillna(series.groupby(series.index.month).transform('median'))

    for col in numeric_cols:
        resampled_df[col] = resampled_df.groupby(resampled_df.index.month)[col].transform(seasonal_interpolation)

        if col in ['TMAX', 'TMIN', 'TAVG']:
            resampled_df[col] = resampled_df[col].fillna(resampled_df[col].rolling(window=15, center=True, min_periods=1).median())

        elif col == 'PRCP':
            log_interpolated = np.log1p(resampled_df[col] + 0.01).interpolate(method='cubic', limit=10)
            resampled_df[col] = np.maximum(np.expm1(log_interpolated), 0)

        resampled_df[col] = resampled_df[col].ffill(limit=60).bfill(limit=60)

        global_median = station_df[col].median()
        resampled_df[col] = resampled_df[col].fillna(global_median)

    for col in numeric_cols:
        if col in ['TMAX', 'TMIN', 'TAVG']:
            Q1, Q3 = resampled_df[col].quantile([0.25, 0.75])
            IQR = Q3 - Q1
            lower_bound, upper_bound = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
            resampled_df[col] = np.where((resampled_df[col] < lower_bound) | (resampled_df[col] > upper_bound), global_median, resampled_df[col])

        elif col == 'PRCP':
            resampled_df[col] = np.maximum(resampled_df[col], 0)

    for key, value in station_meta.items():
        resampled_df[key] = value

    resampled_df = resampled_df.reset_index().rename(columns={'index': 'DATE'})
    resampled_df['MONTH'], resampled_df['DAY'], resampled_df['YEAR'] = resampled_df['DATE'].dt.month, resampled_df['DATE'].dt.day, resampled_df['DATE'].dt.year
    resampled_df['DAY_OF_YEAR'] = resampled_df['DATE'].dt.dayofyear
    resampled_df['SEASON'] = ((resampled_df['MONTH'] % 12) // 3 + 1).astype(int)

    if 'TMAX' in resampled_df.columns and 'TMIN' in resampled_df.columns:
        resampled_df['TEMP_RANGE'] = resampled_df['TMAX'] - resampled_df['TMIN']

    # Final NaN check and global median replacement
    for col in numeric_cols:
        if resampled_df[col].isna().any():
            # print(f"Warning: Remaining NaNs in {col} column")
            global_median = station_df[col].median()
            resampled_df[col] = resampled_df[col].fillna(global_median)


    return resampled_df

# Create metadata by applying function to the first partition
meta_df = preprocess(ddf.partitions[0].head())

In [ ]:
# 4. Apply preprocessing with metadata
print("Preprocessing data...")
ddf = ddf.map_partitions(preprocess, meta=meta_df)

In [ ]:
# 5. Convert Dask dataframe to Pandas for further processing
# This will load all data into memory (ensure you have enough RAM)
print("Converting to pandas dataframe...")
df = ddf.compute()

target_column = 'TMAX'

# Drop rows with NaN values
print("Dropping rows with NaN values...")
print(f"Total rows: {len(df)}")
df = df.dropna(subset=[target_column])
print(f"Rows remaining after dropping NaNs: {len(df)}")

In [ ]:
# 6. Create a time-indexed dataset
# Group by station and apply resampling
print("Resampling data by station...")
processed_stations = []
for station_id, station_df in df.groupby('STATION'):
    try:
        processed_station = process_station_data(station_df)
        if processed_station is not None:
            processed_stations.append(processed_station)
            print(f"Processed station {station_id} with {len(processed_station)} days of data")
    except Exception as e:
        print(f"Error processing station {station_id}: {e}")

# Combine all processed stations
print("Combining processed stations...")
if processed_stations:
    processed_df = pd.concat(processed_stations)
    print(f"Combined dataset has {len(processed_df)} rows")
else:
    raise ValueError("No stations were successfully processed")

In [ ]:
# 7. Split data into training/validation/test sets based on time
print("Splitting data into train/validation/test sets...")

# For each station, perform temporal split
train_data = {}
val_data = {}
test_data = {}

def split_time_series(processed_df, train_ratio=0.7, val_ratio=0.15):
    """
    Split dataframe by station into train/val/test sets based on time,
    preserving proper date information.
    """
    train_data = {}
    val_data = {}
    test_data = {}

    for station_id, station_df in processed_df.groupby('STATION'):
        # Ensure proper datetime column
        if not pd.api.types.is_datetime64_any_dtype(station_df['DATE']):
            station_df['DATE'] = pd.to_datetime(station_df['DATE'])

        # Sort by date
        station_df = station_df.sort_values('DATE')

        # Calculate split points
        n = len(station_df)
        train_end = int(n * train_ratio)
        val_end = int(n * (train_ratio + val_ratio))

        # Split the data
        train = station_df.iloc[:train_end].copy()
        val = station_df.iloc[train_end:val_end].copy()
        test = station_df.iloc[val_end:].copy()

        # Store in dictionaries
        train_data[station_id] = train
        val_data[station_id] = val
        test_data[station_id] = test

    return train_data, val_data, test_data

train_data, val_data, test_data = split_time_series(processed_df)

In [ ]:
# 8. Prepare data for GluonTS format
# First, let's convert the data into the appropriate format for each split
print("Converting data to GluonTS format...")

def prepare_gluonts_data(data_dict, target_column='TMAX'):
    """
    Convert dictionary of dataframes to GluonTS dataset format.

    Args:
        data_dict: Dictionary of dataframes, keyed by station_id
        target_column: Column to use as target variable

    Returns:
        List of dictionaries in GluonTS format
    """
    gluonts_data = []

    # Create a mapping of station IDs to unique categorical indices
    station_ids = list(data_dict.keys())
    station_id_to_cat = {station_id: idx for idx, station_id in enumerate(station_ids)}

    for station_id, df in data_dict.items():
        # Ensure the dataframe has a proper date index or DATE column
        if 'DATE' in df.columns:
            # If DATE is a column, ensure it's a datetime
            if not pd.api.types.is_datetime64_any_dtype(df['DATE']):
                df['DATE'] = pd.to_datetime(df['DATE'])

            # Sort by DATE
            df = df.sort_values('DATE')

            # Get start timestamp as a proper pandas Timestamp
            start = df['DATE'].iloc[0]
        else:
            # If using an index, ensure it's a datetime index
            if not isinstance(df.index, pd.DatetimeIndex):
                df.index = pd.to_datetime(df.index)

            # Sort by date index
            df = df.sort_index()

            # Get start timestamp
            start = df.index[0]

        # Convert start to string format that GluonTS can handle
        # Format: YYYY-MM-DD HH:MM:SS
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')

        # Extract target variable
        target = df[target_column].values.astype(np.float32)

        # Create static categorical feature
        static_cat = [station_id_to_cat[station_id]]

        # Create optional static real features
        static_real = [
            df['LATITUDE'].iloc[0],
            df['LONGITUDE'].iloc[0],
            df['ELEVATION'].iloc[0]
        ]

        # Create dictionary in GluonTS format
        ts_data = {
            "start": start_str,  # Store as string for consistent handling
            "target": target,
            "feat_static_cat": static_cat,
            "feat_static_real": static_real,
            "item_id": station_id
        }

        gluonts_data.append(ts_data)

    return gluonts_data

# Choose target variable
#target_column = 'TMIN'  # Can be changed to TMIN, PRCP, etc.

# Convert data to GluonTS format
train_gluonts = prepare_gluonts_data(train_data, target_column)
val_gluonts = prepare_gluonts_data(val_data, target_column)
test_gluonts = prepare_gluonts_data(test_data, target_column)

print(f"Created {len(train_gluonts)} training series, {len(val_gluonts)} validation series, {len(test_gluonts)} test series")


In [ ]:
# 9. Create HuggingFace Datasets
print("Creating HuggingFace Datasets...")

# Process start field for HuggingFace Dataset
class ProcessStartField:
    ts_id = 0

    def __call__(self, data):
        # Convert start from string to datetime
        if isinstance(data["start"], str):
            data["start"] = pd.Timestamp(data["start"])

        # Ensure start is a timestamp
        if isinstance(data["start"], pd.Timestamp):
            data["start"] = data["start"].to_pydatetime()

        # Assign a unique ID to each time series
        if "feat_static_cat" not in data:
            data["feat_static_cat"] = [self.ts_id]

        self.ts_id += 1
        return data

# Apply processing to each split
process_start = ProcessStartField()
train_processed = list(Map(process_start, train_gluonts))
process_start = ProcessStartField()  # Reset ID counter
val_processed = list(Map(process_start, val_gluonts))
process_start = ProcessStartField()  # Reset ID counter
test_processed = list(Map(process_start, test_gluonts))

# Define dataset features
features = Features(
    {
        "start": Value("timestamp[s]"),
        "target": Sequence(Value("float32")),
        "feat_static_cat": Sequence(Value("int64")),
        "feat_static_real": Sequence(Value("float32")),
        "item_id": Value("string"),
    }
)

# Create HuggingFace Datasets
train_dataset = Dataset.from_list(train_processed, features=features)
val_dataset = Dataset.from_list(val_processed, features=features)
test_dataset = Dataset.from_list(test_processed, features=features)


In [ ]:
# 10. Save the prepared datasets
print("Saving prepared datasets...")
data_dir = "/content/drive/MyDrive/weather_forecasting_project/prepared_datasets"
if os.path.exists(data_dir):
    shutil.rmtree(data_dir)
os.makedirs(data_dir, exist_ok=True)

# Create a DatasetDict
dataset = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})

# Save the DatasetDict
dataset.save_to_disk(f"{data_dir}/dataset")

# Save target column information
with open(f"{data_dir}/metadata.txt", "w") as f:
    f.write(f"target_column={target_column}\n")
    f.write("freq=D\n")  # Daily frequency

print("Data preparation complete!")
print("Datasets saved to:", data_dir)
print("Final dataset sizes:")
print(f"Train: {len(train_dataset)} series")
print(f"Validation: {len(val_dataset)} series")
print(f"Test: {len(test_dataset)} series")

In [ ]:
train_dataset[0]["item_id"]

In [ ]:
print(pd.Timestamp(train_dataset[0]["start"]))
print(pd.Timestamp(val_dataset[0]["start"]))
print(pd.Timestamp(test_dataset[0]["start"]))

In [ ]:
def plot_sequential_time_series(train_example, validation_example, test_example,
                             title=None, target_column="TMAX"):
    """
    Plot time series data sequentially from train, validation, and test sets with datetime on x-axis.
    """
    # Create a figure and axis
    plt.figure(figsize=(15, 6))

    # Extract target values
    train_target = train_example["target"]
    validation_target = validation_example["target"]
    test_target = test_example["target"]

    # Get start dates - parse with pandas to ensure proper handling
    train_start = pd.Timestamp(train_example["start"])
    val_start = pd.Timestamp(validation_example["start"])
    test_start = pd.Timestamp(test_example["start"])

    # Debugging info
    print(f"Train start: {train_start}, length: {len(train_target)}")
    print(f"Val start: {val_start}, length: {len(validation_target)}")
    print(f"Test start: {test_start}, length: {len(test_target)}")

    # Create date ranges for each dataset
    train_dates = pd.date_range(start=train_start, periods=len(train_target), freq='D')
    val_dates = pd.date_range(start=val_start, periods=len(validation_target), freq='D')
    test_dates = pd.date_range(start=test_start, periods=len(test_target), freq='D')

    # Plot each array with actual dates
    plt.plot(train_dates, train_target, color="blue", label="Train")
    plt.plot(val_dates, validation_target, color="red", label="Validation")
    plt.plot(test_dates, test_target, color="green", label="Test")

    # Add labels and title
    plt.xlabel("Date")
    plt.ylabel(f"{target_column} Temperature")
    plt.title(title or f"Time Series Plot of {target_column} ({train_example['item_id']})")
    plt.legend()

    # Add vertical lines to separate datasets
    if len(train_dates) > 0:
        plt.axvline(x=train_dates[-1], color='gray', linestyle='--', alpha=0.7)
    if len(val_dates) > 0:
        plt.axvline(x=val_dates[-1], color='gray', linestyle='--', alpha=0.7)

    # Format x-axis dates
    plt.gcf().autofmt_xdate()

    # Show the plot
    plt.tight_layout()
    plt.show()

# Example usage in your existing code:
for x in range(max_files // 10):
    plot_sequential_time_series(
        train_dataset[x],
        val_dataset[x],
        test_dataset[x],
        target_column=target_column
    )